## Parameters
- model = SEPPexp: simplified SEPP (with 'self-excitation' only in time)
- train= week (2018-07-01 to 2018-07-07)
- prediction_horizon = one day
- validation = nested-cross; day-forward chaining
- db = SIEDCO
- spatial scope: city

## Hypothesis:
Average hit rate using SEPP exp model is higher compared to base line model (experiment_03_counting_city_siedco)


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append("..")
from services.prediction_experiment import PredictionExperiment

Failed to import `rtree`.
Failed to import `rtree`.


In [2]:
def get_value(row,c):
    return row['Hit-rates'][c]

In [3]:
filter_localidad = {'field':'','value':''}
train_dates = {'initial':'2018-07-01','final':'2018-07-01'}
validation_dates = {'initial':'2018-07-02','final':'2018-07-07'}
model = "SEPPexp"
metrics = ''
aggregation = ''

## SIEDCO

In [4]:
csv_path = '/Users/anamaria/Desktop/dev/security_project/datasets/deduplicate_siedco_10032020.csv'
siedco_info = {'name':'SIEDCO','path':csv_path}

In [5]:
city_experiment = PredictionExperiment(dataset_info=siedco_info, custom_filter=filter_localidad,train_dates=train_dates, validation_dates=validation_dates, model=model,metrics='',aggregation_data='')
accuracy_array = city_experiment.run_ncv_experiment(time_unit='',grid_size=150, outer_iterations='')

train_subset_dates {'initial': datetime.datetime(2018, 7, 1, 0, 0), 'final': datetime.datetime(2018, 7, 1, 0, 0)}
current_validation_date 2018-07-02 00:00:00
75


/Users/anamaria/Desktop/dev/security_project/security_venv/lib/python3.7/site-packages/open_cp/seppexp.py:183: RuntimeWarning: invalid value encountered in double_scalars
  omega = _np.sum(upper_trianglar_sums) / _np.sum(weighted_upper_trianglar_sums)
/Users/anamaria/Desktop/dev/security_project/security_venv/lib/python3.7/site-packages/open_cp/seppexp.py:184: RuntimeWarning: invalid value encountered in double_scalars
  theta = _np.sum(upper_trianglar_sums) / _np.sum(event_counts)


Exception: Convergence failed!

In [ ]:
flat_list = [item for sublist in accuracy_array for item in sublist]
df_result = pd.DataFrame(flat_list, columns =['Prediction-time', 'Hit-rates'])
coverages = [2.0,4.0,6.0,8.0,10.0,12.0,14.0,16.0,18.0,20.0]
for c in coverages:
    column = 'coverage_'+str(c)
    df_result[column] = df_result.apply(lambda row: get_value(row,c), axis=1)

In [ ]:
df_siedco = df_result.copy()

In [ ]:
#delete datetimes where no crimes were reported (0 crimes on ground truth -> hit-rate = -1)
df_siedco = df_siedco[df_siedco['coverage_2.0']!= -1]
df_siedco.mean()

In [ ]:
df_siedco['date'] = pd.to_datetime(df_siedco['Prediction-time']).dt.date
df_by_date = df_siedco.groupby(['date']).mean().reset_index()

In [ ]:
# gca stands for 'get current axis'
ax = plt.gca()

df_by_date.plot(kind='line',x='date',y='coverage_2.0',ax=ax)
df_by_date.plot(kind='line',x='date',y='coverage_4.0', ax=ax)
df_by_date.plot(kind='line',x='date',y='coverage_10.0', ax=ax)
df_by_date.plot(kind='line',x='date',y='coverage_20.0', ax=ax)
ax.set_ylabel('hit rate')

plt.show()

## Coverage vs hit rates

In [ ]:
ax = plt.gca()
plt.plot(coverages,df_siedco.mean(),'.',label='SIEDCO')
legend = ax.legend(loc='best')
ax.set_ylabel('hit rate')
ax.set_xlabel('coverage (%)')
plt.ylim(0, 1)
plt.show()

In [ ]:
from services.process_data import ProcessData
path ='/Users/anamaria/Desktop/dev/security_project/aggressive_behavior_model/pkl/'
file_name = 'coverage_vs_hitrate_city_SEPPexp'
ProcessData.save_element(path,file_name,df_siedco)